In [1]:
!pip install -U lazypredict scikit-learn pandas numpy

  Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached pandas-3.0.1-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.4.2-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached protobuf-6.33.5-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached sqlparse-0.5.5-py3-none-any.whl.metadata (4.7 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl (8.1 MB)
   --------------------------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-core 1.5.0 requires scikit-learn<1.8.0,>=1.4.0, but you have scikit-learn 1.8.0 which is incompatible.
autogluon-features 1.5.0 requires scikit-learn<1.8.0,>=1.4.0, but you have scikit-learn 1.8.0 which is incompatible.
autogluon-tabular 1.5.0 requires scikit-learn<1.8.0,>=1.4.0, but you have scikit-learn 1.8.0 which is incompatible.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.8.0 which is incompatible.
pycaret 3.3.2 requires pandas<2.2.0, but you have pandas 2.3.3 which is incompatible.
sktime 0.26.0 requires pandas<2.2.0,>=1.1, but you have pandas 2.3.3 which is incompatible.
sktime 0.26.0 requires scikit-learn<1.5.0,>=0.24, but you have scikit-learn 1.8.0 which is incompatible.


In [2]:
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [3]:
# 1) Load
df = pd.read_csv("ckd.csv")
df.head()

,PatientID,Age,Gender,Ethnicity,SocioeconomicStatus,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,...,Itching,QualityOfLifeScore,HeavyMetalsExposure,OccupationalExposureChemicals,WaterQuality,MedicalCheckupsFrequency,MedicationAdherence,HealthLiteracy,Diagnosis,DoctorInCharge
0,1,71,0,0,0,2,31.07,1,5.13,1.68,...,7.56,76.08,0,0,1,1.02,4.97,9.87,1,Confidential
1,2,34,0,0,1,3,29.69,1,18.61,8.38,...,6.84,40.13,0,0,0,3.92,8.19,7.16,1,Confidential
2,3,80,1,1,0,1,37.39,1,11.88,9.61,...,2.14,92.87,0,1,1,1.43,7.62,7.35,1,Confidential
3,4,40,0,2,0,1,31.33,0,16.02,0.41,...,7.08,90.08,0,0,0,3.23,3.28,6.63,1,Confidential
4,5,43,0,1,1,2,23.73,0,7.94,0.78,...,3.55,5.26,0,0,1,0.29,3.85,1.44,1,Confidential


In [4]:
# 2) Drop unused columns
df = df.drop(columns=["PatientID", "DoctorInCharge"], errors="ignore")

In [5]:
# 3) Target
target = "Diagnosis"
y = df[target]
X = df.drop(columns=[target])

In [6]:
# 4) Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y if y.nunique() > 1 else None
)

In [8]:
# 5) LazyPredict
clf = LazyClassifier(
    verbose=0,
    ignore_warnings=True,
    custom_metric=None
)

models, predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models.head(20))

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 1219, number of negative: 108
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001217 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7933
[LightGBM] [Info] Number of data points in the train set: 1327, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.918613 -> initscore=2.423655
[LightGBM] [Info] Start training from score 2.423655
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
                             Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                         
NearestCentroid                  0.75               0.76     0.76      0.80   
AdaBoostClassifier               0.93               0.62     0.62      0.91   
Perceptron                       0.86               0.62     0.62      0.87   
GaussianNB                       0.92     

In [9]:
models.sort_values("F1 Score", ascending=False).head(20)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,0.93,0.62,0.62,0.91,0.48
GaussianNB,0.92,0.62,0.62,0.90,0.01
LogisticRegression,0.92,0.60,0.60,0.90,0.06
LGBMClassifier,0.92,0.57,0.57,0.90,0.52
XGBClassifier,0.91,0.60,0.60,0.90,0.20
LinearDiscriminantAnalysis,0.92,0.57,0.57,0.90,0.08
BaggingClassifier,0.91,0.59,0.59,0.90,0.32
LinearSVC,0.92,0.54,0.54,0.89,0.03
CalibratedClassifierCV,0.92,0.52,0.52,0.89,0.11


In [10]:
top_by_accuracy = models.sort_values("Accuracy", ascending=False).head(1)
print(top_by_accuracy)

best_model_name = top_by_accuracy.index[0]
print("Top model based on Accuracy:", best_model_name)

                    Accuracy  Balanced Accuracy  ROC AUC  F1 Score  Time Taken
Model                                                                         
AdaBoostClassifier      0.93               0.62     0.62      0.91        0.48
Top model based on Accuracy: AdaBoostClassifier
